In [1]:
import requests
from lxml import html
import pandas as pd
from datetime import date


### pandas

In [ ]:
df = pd.DataFrame(columns=['url_detail_id','scrapped_day','opening_date','closing_date','location'])

In [ ]:
df.to_csv('idb_ops.csv', index=False, encoding='utf-8', header=True)

In [ ]:
del df

### funciones

In [3]:
def get_page(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    return tree

### primer pagina


In [3]:
#lee la tabla de la pagina
historicals = pd.read_csv('idb_ops.csv', encoding='utf-8')

In [4]:
historicals

,url_detail_id,scrapped_day,opening_date,closing_date,location


In [ ]:
historicals['url_detail_id'][historicals['url_detail_id']=='']

In [6]:
today = date.today()
today

datetime.date(2023, 3, 11)

In [8]:
hoy = today.strftime("%d/%m/%Y")
hoy

'11/03/2023'

In [30]:
new_row = {
    'url_detail_id':'https://iadbcareers.referrals.selectminds.com/jobs/ite-ipp-devsecops-and-microservices-engineer-consultant-4953' ,
    'scrapped_day': hoy,
    'opening_date': 'Mar 13, 2023',
    'closing_date': 'Mar 27, 2023',
    'location': 'Country Office'}

In [31]:
historicals = historicals.append(new_row, ignore_index=True)

In [53]:
historicals

,url_detail_id,scrapped_day,opening_date,closing_date,location
0,https://iadbcareers.referrals.selectminds.com/...,11/03/2023,"Mar 13, 2023","Mar 27, 2023",Country Office


In [42]:
url_pr1 = 'https://iadbcareers.referrals.selectminds.com/jobs/climate-change-consultant-4955'

In [43]:
url_pr2 = 'https://iadbcareers.referrals.selectminds.com/jobs/ite-ipp-devsecops-and-microservices-engineer-consultant-4953'

In [52]:
if historicals['url_detail_id'][historicals['url_detail_id']==url_pr2].any():
    print('ya existe')

ya existe


In [9]:
arbol = get_page('https://iadbcareers.referrals.selectminds.com/')

In [10]:
print(arbol.xpath('//div[@class="flg_hldr"]/span/text()'))

['New', 'New', 'New', 'New', 'New']


### pagina de detalle

In [11]:
#obtiene los divs que envuelve a cada oportunidad
items = arbol.xpath('//div[contains(@id,"job_list_")]')
items

[<Element div at 0x21efb680db0>,
 <Element div at 0x21efb6534f0>,
 <Element div at 0x21efb653130>,
 <Element div at 0x21efb653e50>,
 <Element div at 0x21efb653ae0>]

In [12]:
item = items[0]
item

<Element div at 0x21efb680db0>

In [13]:
#get the url of detail
detail_url = item.xpath('./div/div/p/a[contains(@href,"/jobs/")]/@href')[0]
detail_url

'https://iadbcareers.referrals.selectminds.com/jobs/ite-ipp-devsecops-and-microservices-engineer-consultant-4953'

In [4]:
detail_page = get_page('https://iadbcareers.referrals.selectminds.com/jobs/treasury-and-risk-%E2%80%93-funding-senior-specialist-4967')

In [15]:
#title
detail_page.xpath('//div[@class="content_header"]/h1/text()')

['\n\t\t\t\t\tITE/IPP DevSecOps and Microservices Engineer Consultant\n\t\t\t\t']

In [83]:
#location
location = detail_page.xpath('//h4[@class="primary_location"]/a/text()')
location

['\n\t\t\t\t\t\t', '\n\t\t\t\t\t\tCountry Office\n\t\t\t\t\t']

In [5]:
import re

In [ ]:
re.sub(r'[\n\t]', '', location)

In [27]:
#Opening Date:
detail_page.xpath('//label[contains(text(),"External Opening Date:")]/text()')[0]\
    .replace('External Opening Date:','').strip()

'Mar 13, 2023'

In [29]:
#Closing Date
detail_page.xpath('///label[contains(text(),"External Closing Date: ")]/text()')[0]\
    .replace('External Closing Date:','').strip()

'Mar 27, 2023'

In [7]:
words_to_look = [
    'Salud',
    'Farmacoeconomía',
    'Medicamentos',
    'Health',
    'Pharmacoeconomics',
    'Medicines',
    'Santé',
    'Pharmacoéconomie',
    'Médicaments',
    'Saude',
    'Farmacoeconomia',
    'Medicamentos'
    ]

In [55]:
detail_page.xpath('//div[@class="job_description"]//descendant::text()')

In [13]:
text_for_alert = ' '.join(detail_page.xpath('//div[@class="job_description"]//descendant::text()'))
text_for_alert = re.sub(r'[\n\t\xa0]', '', text_for_alert ).split("Requirements")[0].strip()
text_for_alert 

'Treasury and Risk – Funding Senior Specialist Headquarters, Washington, D.C.  The IDB Group is a community of diverse, versatile, and passionate people who come together on a journey to improve lives in Latin America and the Caribbean. Our people find purpose and do what they love in an inclusive, collaborative, agile, and rewarding environment.\u202f   About this position  We are looking for a dedicated, energetic, and team-oriented Treasury & Risk-Funding Senior Specialist.  You will work in the Funding team, which funds the Bank by issuing bonds in the international capital markets; executes derivatives in conjunction with the bond issuances; and is responsible for the Investor Relations Program.  What you’ll do:    Work with ESG agencies regarding IDB’s ratings and seek to maintain the highest possible rating standards.     Enhance reporting for ESG debt products, including the EYE Bond and Sustainable Development Bond in part by automating and consolidating information from vario

In [ ]:
text_for_alert = re.sub(r'[\n\t\xa0]', '', text_for_alert ).strip()
text_for_alert 

In [76]:
if any(word in text_for_alert for word in words_to_look):
    print('si')

si


In [80]:
[word in text_for_alert for word in words_to_look]

[False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [73]:
re.findall("Health", text_for_alert)

['Health', 'Health', 'Health']

In [85]:
text_for_alert.lower()

"ite/ipp devsecops and microservices engineer consultant    the idb group is a community of diverse, versatile, and passionate people who come together on a journey to improve lives in latin america and the caribbean. our people find purpose and do what they love in an inclusive, collaborative, agile, and rewarding environment    about this position:  ite/ipp policy and planning division is looking for an experienced professional with background in devsecops, containers and microservices to evaluate, develop and implement a devsecops platforms and a cloud-based container management platform.  the information technology department (ite) is responsible for formulating the bank's it strategy, its implementation and providing related tools, solutions, and services. the ite/ipp policy and planning division is the focal point for it policies, architecture, and governance for the idb.     what you’ll do:   you will report to the enterprise architecture team leader in the information technolog